In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("dreammarket")

create_database('blp_dataset')
create_table('products_tmp')

In [ ]:
original_images = [] # images ok (I wish there were more of these (╥_╥) )

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\market")

    for file in os.listdir(folder):
        if "viewProduct_offer" in file:
            # get text content
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}")).read())

            ## GET DESCRIPTION
            description = ""
            # remove html content, whitespaces and line breaks
            raw_desc = page_content.find("div", attrs={"id": "offerDescription"})

            if raw_desc is not None:
                raw_description = get_content(raw_desc)

            ## GET TITLE
            title = ""
            raw_title = page_content.find("div", attrs={"class": "title"})

            if raw_title is not None:
                title = get_content(raw_title)
 
            ## GET PRICE
            price = ""
            price_text = page_content.find(text="Price")
            
            if price_text is not None:
                price_father = price_text.parent

                if price_father is not None:
                    price_content = price_father.parent

                    if price_content is not None:
                        raw_price = price_content.find("div")
                        
                        if raw_price is not None:
                            price = get_content(raw_price)

            if description != "" or title != "":
                # Insert product into database
                query = "INSERT INTO products_tmp (name, description, price, market_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                values = (title, description, price, "Dream Market", "t")

                cursor.execute(query, values)
                conn.commit()

                product_id = cursor.lastrowid

                ## GET IMAGE
                raw_name = page_content.find("img", attrs={"class": "productImage"})['src']

                if raw_name == "img/no-image.png":
                    query = "UPDATE products_tmp SET has_image = %s WHERE id = %s"
                    cursor.execute(query, ("no_image", product_id))
                    conn.commit()
                else:
                    if raw_name in original_images:
                        query = "UPDATE products_tmp SET has_image = %s WHERE id = %s"
                        cursor.execute(query, ("repeated", product_id))
                        conn.commit()
                    else:
                        original_images.append(raw_name)

                        image_name = re.sub('[?]', '_', raw_name)
                        url_image = Path(f"{folder}\{image_name}")

                        # copy images to new directory, renaming with product id
                        if url_image.exists():
                            dst_dir = Path(f"D:\images\dream_market\{product_id}.jpg")
                            copyfile(url_image, dst_dir)
                        else:
                            query = "UPDATE products_tmp SET has_image = %s WHERE id = %s"
                            cursor.execute(query, ("no_image", product_id))
                            conn.commit()